Access token: NLSQjUEWly7uv5AK1-fwSwXXl0QN9eQSYlJ2DP_0qpCepn1MLaXozB6VSvkqz1qX

In [8]:
import requests
from bs4 import BeautifulSoup
import re

In [47]:
def get_artist_id(artist_name, access_token):
    url = "https://api.genius.com/search"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"q": artist_name}
    
    response = requests.get(url, params=params, headers=headers).json()
    hits = response["response"]["hits"]
    
    for hit in hits:
        if hit["result"]["primary_artist"]["name"].lower() == artist_name.lower():
            return hit["result"]["primary_artist"]["id"]
    return None

def get_artist_songs(artist_id, access_token, max_songs=10):
    songs = []
    page = 1
    headers = {"Authorization": f"Bearer {access_token}"}
    
    while len(songs) < max_songs:
        url = f"https://api.genius.com/artists/{artist_id}/songs"
        params = {"per_page": 20, "page": page, "sort": "popularity"}
        response = requests.get(url, params=params, headers=headers).json()
        new_songs = response["response"]["songs"]
        
        if not new_songs:
            break
        
        for song in new_songs:
            if song["primary_artist"]["id"] == artist_id:
                songs.append({
                    "title": song["title"],
                    "url": song["url"],
                    "release_date": song.get("release_date", "N/A")
                })
                if len(songs) >= max_songs:
                    break
        page += 1
        
    return songs

def get_lyrics_from_url(song_url):
    page = requests.get(song_url)
    soup = BeautifulSoup(page.text, 'html.parser')
    lyrics = []

    # Find the div that contains the lyrics
    lyrics_div = soup.find("div", attrs={"data-lyrics-container": "true"})
    
    # Use .decode_contents() to get the raw HTML (including <br> tags)
    lyric_links = lyrics_div.find_all("a", attrs={"data-ignore-on-click-outside": "true"})
    
    for link in lyric_links:
        # Get the raw HTML content
        lyric_html = link.find("span").decode_contents()

        # Replace <br> tags with a whitespace
        lyric_text = re.sub(r"<br\s*/?>", " ", lyric_html)
        
        # Remove bracketed sections (like [Chorus], [Verse 1], etc.)
        lyric_text = re.sub(r"\[.*?\]", "", lyric_text)
        
        # Remove parentheses (if needed)
        lyric_text = re.sub(r"[()]", "", lyric_text)
        
        lyrics.append(lyric_text)

    # Split the lyrics into words, removing punctuation, and keeping apostrophes
    words = [re.sub(r"[^\w']", '', word) for sentence in lyrics for word in sentence.split()]
    words = [word for word in words if word]  # remove empty strings

    return words
    


def get_kendrick_lyrics(access_token, max_songs=5):
    artist_id = get_artist_id("Kendrick Lamar", access_token)
    if not artist_id:
        return "Artist not found."

    songs = get_artist_songs(artist_id, access_token, max_songs)
    for song in songs:
        print(f"Fetching lyrics for: {song['title']}")
        lyrics = get_lyrics_from_url(song["url"])
        song["lyrics"] = lyrics
    return songs

In [ ]:
access_token = "NLSQjUEWly7uv5AK1-fwSwXXl0QN9eQSYlJ2DP_0qpCepn1MLaXozB6VSvkqz1qX"
kendrick_songs = get_kendrick_lyrics(access_token, max_songs=10)

for song in kendrick_songs:
    print(f"\n{song['title']} ({song['release_date']})\n")
    print(song['lyrics'][:500])  # preview first 500 chars
    print("...\n" + "-"*80)
    print(song["url"])


In [49]:
def get_artist_lyrics(access_token, artist, max_songs=5):
    artist_id = get_artist_id(artist, access_token)
    if not artist_id:
        return "Artist not found."

    songs = get_artist_songs(artist_id, access_token, max_songs)
    for song in songs:
        print(f"Fetching lyrics for: {song['title']}")
        lyrics = get_lyrics_from_url(song["url"])
        song["lyrics"] = lyrics
    return songs

In [50]:
pop_culture_artists = ["Taylor Swift", "Billie Eilish", "Rihanna", "Ariana Grande", "The Weekend", "Kendrick Lamar", "Drake", "Lady Gaga", "Justin Bieber"]

In [51]:
access_token = "NLSQjUEWly7uv5AK1-fwSwXXl0QN9eQSYlJ2DP_0qpCepn1MLaXozB6VSvkqz1qX"
all_songs = []
for artist in pop_culture_artists:
    all_songs.append(get_artist_lyrics(access_token, artist, max_songs=2))

for thing in all_songs:
    for song in thing:
        print(f"\n{song['title']} ({song['release_date']})\n")
        print(song['lyrics'][:500])  # preview first 500 chars
        print("...\n" + "-"*80)
        print(song["url"])


Fetching lyrics for: All Too Well (10 Minute Version) (Taylor’s Version) [From The Vault]
Fetching lyrics for: All Too Well (10 Minute Version) (Taylor’s Version) [Live Acoustic]
Fetching lyrics for: lovely
Fetching lyrics for: when the party’s over
Fetching lyrics for: Work
Fetching lyrics for: Love on the Brain
Fetching lyrics for: thank u, next
Fetching lyrics for: 7 rings
Fetching lyrics for: Not Like Us
Fetching lyrics for: HUMBLE.
Fetching lyrics for: God’s Plan
Fetching lyrics for: In My Feelings
Fetching lyrics for: Shallow
Fetching lyrics for: Die With A Smile
Fetching lyrics for: Love Yourself
Fetching lyrics for: Baby

All Too Well (10 Minute Version) (Taylor’s Version) [From The Vault] (N/A)

['I', 'walked', 'through', 'the', 'door', 'with', 'you', 'the', 'air', 'was', 'cold', 'But', "somethin'", "'bout", 'it', 'felt', 'like', 'home', 'somehow', 'And', 'I', 'left', 'my', 'scarf', 'there', 'at', 'your', "sister's", 'house', 'And', "you've", 'still', 'got', 'it', 'in', 'your'

TypeError: string indices must be integers, not 'str'